# Import Libraries and Environment Variables

In [1]:
import os
import json
import base64
from dotenv import load_dotenv
from httpayer import HTTPayerClient

In [2]:
load_dotenv()

True

# Initialize the Client

In [ ]:
EVM_PRIVATE_KEY = os.getenv("EVM_PRIVATE_KEY")

if not EVM_PRIVATE_KEY:
    raise ValueError("EVM_PRIVATE_KEY environment variable is not set.")

client = HTTPayerClient(
    private_key=EVM_PRIVATE_KEY,
    network="base" # Network to pay on
)

# Helper Functions

In [4]:
def decode_payment_response(header_value: str) -> dict:
    """Decode x-payment-response header (base64 encoded JSON)"""
    try:
        decoded = base64.b64decode(header_value).decode('utf-8')
        return json.loads(decoded)
    except Exception as e:
        print(f"Warning: Could not decode payment response: {e}")
        return {"raw": header_value}


def extract_payment_info(response):
    """Extract and decode payment headers from response"""
    payment_info = {}

    # Extract x-client-payment (client's tx to HTTPayer)
    if "x-client-payment" in response.headers:
        payment_info["client_payment"] = response.headers["x-client-payment"]
        print(f"[Payment] Client TX: {payment_info['client_payment']}")

    # Extract x-payment-response (HTTPayer's tx to target API)
    payment_response_header = (
        response.headers.get("payment-response") or
        response.headers.get("x-payment-response")
    )

    if payment_response_header:
        decoded = decode_payment_response(payment_response_header)
        payment_info["payment_response"] = decoded
        print(f"[Payment] HTTPayer TX: {json.dumps(decoded, indent=2)}")

    return payment_info

# Relay Limits



In [5]:
relay_limits = client.get_relay_limits()
print("Relay limits:", json.dumps(relay_limits, indent=2))

Relay limits: {
  "wallet_address": "8yfqcp3btr4zt2wfx6hmgdmdpxoxnx8ewcfvajl1mjwu",
  "mainnet": {
    "wallet_limits": {
      "daily_limit": 100,
      "today_spent": 0,
      "remaining": 100,
      "percentage_used": "0.00"
    },
    "system_limits": {
      "daily_limit": 5000,
      "today_spent": 0.6283,
      "remaining": 4999.3717,
      "percentage_used": "0.01"
    },
    "can_relay": true
  },
  "testnet": {
    "wallet_limits": {
      "daily_limit": 100,
      "today_spent": 0,
      "remaining": 100,
      "percentage_used": "0.00"
    },
    "system_limits": {
      "daily_limit": 5000,
      "today_spent": 0,
      "remaining": 5000,
      "percentage_used": "0.00"
    },
    "can_relay": true
  },
  "relay_fee_model": {
    "percentage": 3,
    "minimum_usd": 0.002,
    "description": "3% with $0.002 minimum",
    "formula": "Fee = max(percentage \u00d7 target_amount, minimum_fee)"
  },
  "reset_time": "2026-01-13T00:00:00.000Z"
}


# Step 1: Get Gloria AI news

AI powered News & Data 

More info: [https://www.itsgloria.ai/](https://www.itsgloria.ai/)

In [6]:
def get_gloria_news():
    """Fetch AI and crypto news from Gloria AI"""
    print("[Step 1] Fetching Gloria AI news...")

    GLORIA_AI_URL = "https://api.itsgloria.ai/news?feed_categories=ai,crypto"

    response = client.request(
        method="GET",
        url=GLORIA_AI_URL,
        headers={"Accept": "application/json"}
    )

    response.raise_for_status()

    # Extract payment info
    print()
    payment_info = extract_payment_info(response)
    print()

    data = response.json()
    print(f"[Gloria] Received {len(data)} news items")

    # Parse news summaries
    gloria_news_summaries = {}
    for item in data:
        signal = item.get("signal")
        sentiment = item.get("sentiment")
        summary = item.get("short_context") or "No summary available."
        source = item.get("sources")[0] if item.get("sources") else "Unknown"

        gloria_news_summaries[signal] = {
            "sentiment": sentiment,
            "summary": summary,
            "source": source,
        }

    return gloria_news_summaries, payment_info

In [7]:
gloria_summaries, gloria_payment = get_gloria_news()

[Step 1] Fetching Gloria AI news...

[Payment] Client TX: 1cMVTpMn5noXF8fvqixs2jJHgte1UL8T3UZphbtL6KqZkfQGwndawu2W13eswCYsBG4fyNbEv2V8rhejm5rEbov
[Payment] HTTPayer TX: {
  "network": "base",
  "payer": "0xb2F1C3bEb9C4c4bb7fE8E55a9751603C2a3c0e54",
  "success": true,
  "transaction": "0xa8a27533b261d12849c681c941d92ffdfa637c2a1ddfdc7e0c48c4e428b01da2"
}

[Gloria] Received 10 news items


# Step 2: Get Nansen smart money data

A curated list of the top 5,000 highest-performing wallets ranked by realised profit, winrate, and strong performance across market cycles.
Netflow data gets aggregated token flow analysis for smart money wallets to see which tokens they're accumulating or distributing.

More info: [https://docs.nansen.ai/api/smart-money](https://docs.nansen.ai/api/smart-money)

In [8]:
def get_nansen_data():
    """Fetch smart money netflow data from Nansen"""
    print("[Step 2] Fetching Nansen smart money data...")

    NANSEN_API_URL = "https://nansen.api.corbits.dev/api/v1/smart-money/netflow"

    request_data = {
        "chains": ["ethereum", "solana"],
        "filters": {
            "exclude_smart_money_labels": ["30D Smart Trader"],
            "include_native_tokens": False,
            "include_smart_money_labels": ["Fund", "Smart Trader"],
            "include_stablecoins": True
        },
        "pagination": {
            "page": 1,
            "per_page": 10
        },
        "order_by": [
            {
                "field": "net_flow_30d_usd",
                "direction": "DESC"
            }
        ]
    }

    response = client.request(
        method="POST",
        url=NANSEN_API_URL,
        headers={"Content-Type": "application/json"},
        json=request_data
    )

    response.raise_for_status()

    # Extract payment info
    print()
    payment_info = extract_payment_info(response)
    print()

    nansen_data = response.json()

    # Extract tokens and sectors
    token_symbols = [item['token_symbol'] for item in nansen_data.get("data", [])]
    sectors = set()
    filtered_nansen_data = {}

    print(f"[Nansen] Received {len(nansen_data.get('data', []))} smart money flows:")
    for item in nansen_data.get("data", []):
        print(f"  • {item['token_symbol']}: ${item['net_flow_30d_usd']:,.2f} | {', '.join(item['token_sectors'])}")
        for sector in item['token_sectors']:
            sectors.add(sector)

        filtered_nansen_data[item['token_symbol']] = {
            "net_flow_30d_usd": item['net_flow_30d_usd'],
            "token_sectors": item['token_sectors']
        }

    return filtered_nansen_data, token_symbols, sectors, payment_info

In [9]:
nansen_data, token_symbols, sectors, nansen_payment = get_nansen_data()

[Step 2] Fetching Nansen smart money data...

[Payment] Client TX: 0x65ec20cbb95483a26f9c081c6e18e7ae87c158eba88bbe32d4c7249ade6dd7ed
[Payment] HTTPayer TX: {
  "success": true,
  "txHash": "VGTMqjAEx1frvUiL4TDZnRVkGDf8r4sxJMz8m4sA4SSvjo9Kq9hxns1HBwvXjzTkAvfVxiySVii6XdXty5TUrnR",
  "networkId": "solana-mainnet-beta",
  "amount": 1e-08,
  "timestamp": "2026-01-12T13:58:40.317Z"
}

[Nansen] Received 10 smart money flows:
  • USDT: $28,354,896.79 | Crypto Payments, Stablecoin
  • SYRUP: $2,459,524.07 | DeFi Lending (Money Markets), Yield Bearing
  • XAUT: $1,999,631.53 | Stablecoin
  • PAXG: $1,999,631.53 | RWAs, Stablecoin
  • USDS: $500,493.37 | Stablecoin
  • MOCA: $404,571.69 | Crypto Payments, NFTs
  • BGB: $382,772.95 | 
  • APE: $262,723.73 | NFTs, GameFi, Metaverse
  • HSK: $258,521.90 | Scaling & Connectivity
  • GRASS: $225,356.80 | Artificial Intelligence, Scaling & Connectivity, Data source and Oracle


# Step 3: Get Heurist news

Real-time AI search engine

More info: [https://docs.heurist.ai/introduction](https://docs.heurist.ai/introduction)

In [10]:
def get_heurist_search(token_symbols, sectors):
    """Search for crypto news using Heurist AI based on Nansen tokens"""
    print("[Step 3] Searching crypto news with Heurist AI...")

    HEURIST_API_URL = "https://mesh.heurist.xyz/x402/agents/ExaSearchDigestAgent/exa_web_search"

    search_term = f"Recent cryptocurrency news and market analysis for tokens: {', '.join(token_symbols)}. Focus on {', '.join(sectors)} sectors."
    print(f"[Heurist] Search term: {search_term[:100]}...")

    data = {
        "search_term": search_term,
        "limit": 5,
        "time_filter": "past_month"
    }

    response = client.request(
        method="POST",
        url=HEURIST_API_URL,
        headers={"Content-Type": "application/json"},
        json=data
    )

    response.raise_for_status()

    # Extract payment info
    print()
    payment_info = extract_payment_info(response)
    print()

    heurist_data = response.json().get("result", {}).get("data", {})
    print(f"[Heurist] Search results received")

    return heurist_data, payment_info

In [11]:
heurist_data, heurist_payment = get_heurist_search(token_symbols, list(sectors))

[Step 3] Searching crypto news with Heurist AI...
[Heurist] Search term: Recent cryptocurrency news and market analysis for tokens: USDT, SYRUP, XAUT, PAXG, USDS, MOCA, BGB,...

[Payment] Client TX: 0x2ede9ef23c3be61edc142f2adcd795418319253332bef5c75a7814faffa4dde2
[Payment] HTTPayer TX: {
  "success": true,
  "transaction": "0x503551ceab5f39bba8573e7b17007ca10bdeb34dd6c3346c816e55339aefca95",
  "network": "base",
  "payer": "0xb2F1C3bEb9C4c4bb7fE8E55a9751603C2a3c0e54"
}

[Heurist] Search results received


# Step 4: Summarize with LLM

This calls HTTPayer's hosted LLM endpoint

In [12]:
def summarize_with_llm(gloria_data, nansen_data, heurist_data, temperature=0.7):
    """Generate investment insights using LLM based on all gathered data"""
    print("[Step 4] Generating investment analysis with LLM...")

    messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant that analyzes cryptocurrency smart money flows and related news to provide actionable insights."
        },
        {
            "role": "user",
            "content": f"""Analyze the following data:

GLORIA AI NEWS:
{json.dumps(gloria_data, indent=2)}

SMART MONEY TOKEN FLOWS (from Nansen):
{json.dumps(nansen_data, indent=2)}

RELATED NEWS & MARKET ANALYSIS (from Heurist):
{json.dumps(heurist_data, indent=2)}

Provide a concise summary that:
1. Identifies which tokens smart money is accumulating or selling
2. Explains potential reasons based on the news articles
3. Highlights key trends or opportunities
4. Suggests how an investor should position their portfolio based on these insights"""
        }
    ]

    url = "https://api.httpayer.com/llm/chat"

    payload = {
        "messages": messages,
        "temperature": temperature
    }
    headers = {"Content-Type": "application/json"}

    direct_client = HTTPayerClient(
        private_key=EVM_PRIVATE_KEY,
        network="base", # Network to pay on
        privacy_mode=False # Direct x402 payment
    )

    response = direct_client.request(
        method="POST",
        url=url,
        headers=headers,
        json=payload
    )

    response.raise_for_status()

    # Extract payment info
    print()
    payment_info = extract_payment_info(response)
    print()

    llm_data = response.json()
    print("[LLM] Investment analysis generated")

    return llm_data, payment_info

In [13]:
llm_response, llm_payment = summarize_with_llm(
    gloria_summaries,
    nansen_data,
    heurist_data,
    temperature=0.7
)

[Step 4] Generating investment analysis with LLM...

[Payment] HTTPayer TX: {
  "success": true,
  "payer": "0xeb1d7c1b6cCc8b48bC764A9C0c5d68E081F0aBEF",
  "transaction": "0x99e3fb4cad15ddcd87b51dfc5e98dd8b9ec4807ec3f95635df275479f64f1b1a",
  "network": "base"
}

[LLM] Investment analysis generated


In [37]:
print("LLM Investment Analysis:")
print(llm_response.get("response", ""))

LLM Investment Analysis:
### Summary of Smart Money Flows and Market Insights

#### 1. Smart Money Accumulation
- **Accumulated Tokens:**
  - **USDT (Tether):** $27.1 million net inflow in 30 days.
  - **SYRUP:** $2.46 million net inflow.
  - **XAUT (Tether Gold):** $1.99 million net inflow.
  - **PAXG (Paxos Gold):** $1.99 million net inflow.
  
- **Tokens with Minimal Activity:**
  - **USDS, MOCA, BGB, APE, HSK, GRASS:** These tokens have lower net inflows, indicating less interest or speculative activity.

#### 2. Reasons for Accumulation
- **USDT:** Continues to dominate as a preferred stablecoin for transactions, reflecting its use as a digital store of value amidst increasing stablecoin transaction volumes and market expansion.
- **XAUT and PAXG:** The rise in gold prices and interest in tokenized assets are driving demand for gold-backed stablecoins. The introduction of Tether's "Scudo" fractional unit for XAUT facilitates easier transactions and is likely attracting investors s